### 作業目標: 了解N-Gram如何藉由文本計算機率

### 為何需要使用馬可夫假設來簡化語言模型的計算?

原本的語言模型利用貝氏定理計算機率時為:
$$
\begin{aligned}
&W = (W_1W_2W_3W_4…W_m) \\
&P(W_1,W_2,W_3,W_4,…,W_m) = P(W_1)*P(W_2|W_1)*P(W_3|W_1,W_2)*…*P(W_m|W_1,…,W_{m-1})
\end{aligned}
$$

為何需要引入馬可夫假設使機率簡化為:
$$
P(W_m|W_1,W_2,W_3,…,W_{m-1}) = P(W_m|W_{m-n+1},W_{m-n+2},…,W_{m-1})
$$

答：
馬可夫假設在這裡是利用前一個單字來判斷接下來的單字是否合理。
而一般在判斷句子是否通順時，只需判斷每個單字前後的用是否合理即可。
因此此時選擇用馬可夫假設來簡化機率會較為適當。

### 以Bigram模型下判斷語句是否合理

已知的機率值有
1. p(i|_start_) = 0.25
2. p(english|want) = 0.0011
3. p(food|english) = 0.5
4. p(_end_|food) = 0.68
5. P(want|_start_) = 0.25
6. P(english|i) = 0.0011

In [ ]:
import numpy as np
import pandas as pd
words = ['i', 'want', 'to', 'eat', 'chinese', 'food', 'lunch', 'spend']
word_cnts = np.array([2533, 927, 2417, 746, 158, 1093, 341, 278]).reshape(1, -1)
df_word_cnts = pd.DataFrame(word_cnts, columns=words)
df_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
0,2533,927,2417,746,158,1093,341,278


In [ ]:
# 記錄當前字與前一個字詞的存在頻率
bigram_word_cnts = [[5, 827, 0, 9, 0, 0, 0, 2], [2, 0, 608, 1, 6, 6, 5, 1], [2, 0, 4, 686, 2, 0, 6, 211],
                    [0, 0, 2, 0, 16, 2, 42, 0], [1, 0, 0, 0, 0, 82, 1, 0], [15, 0, 15, 0, 1, 4, 0, 0],
                    [2, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]

df_bigram_word_cnts = pd.DataFrame(bigram_word_cnts, columns=words, index=words)
df_bigram_word_cnts

,i,want,to,eat,chinese,food,lunch,spend
i,5,827,0,9,0,0,0,2
want,2,0,608,1,6,6,5,1
to,2,0,4,686,2,0,6,211
eat,0,0,2,0,16,2,42,0
chinese,1,0,0,0,0,82,1,0
food,15,0,15,0,1,4,0,0
lunch,2,0,0,0,0,1,0,0
spend,1,0,1,0,0,0,0,0


由上表可知當前一個字詞(列)是want的時候, 當前字詞(行)是to的頻率在文本中共有608次

In [ ]:
#請根據給出的總詞頻(df_word_cnts)與bigram模型的詞頻(df_bigram_word_cnts)計算出所有詞的配對機率(ex:p(want|i))
df_bigram_prob = df_bigram_word_cnts.copy()

per = []
for index in range(len(words)):
  per.append([])
  for column in range(len(words)):
    per[index].append(round(bigram_word_cnts[index][column] / word_cnts[0][column], 5))
df_bigram_prob = pd.DataFrame(per, columns=words, index=words)
   
df_bigram_prob

,i,want,to,eat,chinese,food,lunch,spend
i,0.00197,0.89213,0.00000,0.01206,0.00000,0.00000,0.00000,0.00719
want,0.00079,0.00000,0.25155,0.00134,0.03797,0.00549,0.01466,0.00360
to,0.00079,0.00000,0.00165,0.91957,0.01266,0.00000,0.01760,0.75899
eat,0.00000,0.00000,0.00083,0.00000,0.10127,0.00183,0.12317,0.00000
chinese,0.00039,0.00000,0.00000,0.00000,0.00000,0.07502,0.00293,0.00000
food,0.00592,0.00000,0.00621,0.00000,0.00633,0.00366,0.00000,0.00000
lunch,0.00079,0.00000,0.00000,0.00000,0.00000,0.00091,0.00000,0.00000
spend,0.00039,0.00000,0.00041,0.00000,0.00000,0.00000,0.00000,0.00000


請根據已給的機率與所計算出的機率(df_bigram_prob), 試著判斷下列兩個句子哪個較為合理

s1 = “i want english food”

s2 = "want i english food"

答：s1

對前兩個單字而言，
"i want"的機率 (0.89213) 大於 "want i"的機率 (0.00079)
